In [ ]:
import os,datetime
from PIL import Image,ExifTags
import re
import hashlib
from hachoir.parser import createParser
import warnings
from hachoir.metadata import extractMetadata

In [ ]:
def creation_date_of_video(filename):
    warnings.simplefilter("ignore")
    parser = createParser(filename)
    metadata = extractMetadata(parser)
    return metadata.get('creation_date').strftime("%Y_%m_%d %H_%M_%S")

In [ ]:
# Don't call this directly..
def get_exif_date_proxy(file):
    try:
        img = Image.open(file)
        try:
            return img._getexif()[36867]
        except:
            try: 
                return img._getexif()[306]
            except:    
                return None
    except:
         try:
            return creation_date_of_video(file)
         except:
            return None


        
def get_exif_date(file):
    result = get_exif_date_proxy(file)
    if  result is None:
        return None
    else:
        result = re.sub(':', '_', result)
        if re.search('[0-9]{4}_[0-9]{2}_[0-9]{2} [0-9]{2}_[0-9]{2}_[0-9]{2}', result) is None:
            return None
        else:
            dateparsed = datetime.datetime.strptime(result, "%Y_%m_%d %H_%M_%S").year
            if dateparsed < 1991 or dateparsed > 2025:
                print('Tarihten red: ' + file)
                return None
            return result


In [ ]:
def rename_preserving_extension(file, name):
    filename = re.sub('^\.\/', '', str(file))
    correctName = name + re.search('\.([^\.]+)$', filename).group()
    correctName = re.sub('[^0-9A-Z-_\.a-z\/]', '', correctName)
    os.rename(file, correctName);

In [ ]:
def exif_create_folder_for_date(file):
    result = get_exif_date(file)
    result = re.sub(':', '_', result)
    result = re.sub('.{9}$', '', result)
    year = re.search('[0-9]{4}', result).group()
    try:
        os.mkdir(year)
    except:
        None
    try:
        os.mkdir(year + '/' + result)
    except:
        None
    return year + '/' + result


In [ ]:
def exif_get_hour_of_creation(file):
    result = get_exif_date(file)
    result = re.sub(':', '_', result)
    result = re.sub('^.{11}', '', result)
    return result

In [ ]:
def md5sum(filename):
    md5_hash = hashlib.md5()
    with open(filename,"rb") as f:
        # Read and update hash in chunks of 4K
        for byte_block in iter(lambda: f.read(4096),b""):
            md5_hash.update(byte_block)
        return re.search('^.{5}', md5_hash.hexdigest()).group()

In [ ]:
directory = os.fsencode('.')
    
    
for root,d_names,f_names in os.walk('.'):
    if re.search('[0-9]{4}\/[0-9]{4}_[0-9]{2}_[0-9]{2}', str(root)) is None:
        for f in f_names:
            file = os.path.join(root, f)
        
            if get_exif_date(file) is None:
                None # Skip
            else:
                folder = exif_create_folder_for_date(file)
                newname = folder + '/' + exif_get_hour_of_creation(file) + '_' + md5sum(file)
                rename_preserving_extension(file, newname)
                print(file + " -----> " + newname)
print('end')